In [1]:
using Distributed,CairoMakie
using BenchmarkTools
rmprocs(workers())
addprocs(restrict=true)
@everywhere begin
using DifferentialEquations,DiffEqSensitivity
using ForwardDiff, Zygote
end

┌ Warning: rmprocs: process 1 not removed
└ @ Distributed C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.6\Distributed\src\cluster.jl:1038


In [2]:
nprocs()

9

In [3]:
@everywhere begin
    N = 10000000 #community size
    E0 = 10 #init Exposed patients
    beta = 0.00000004 #infection force
    lp = 5.5 #latent period(days)
    ip = 7 #indectious period(days)
    ve = 0.9 #vaccine effectiveness
    vr = 10000 #vaccination rate(per day)
    idr = 365 #immunity duration for R(days)
    idv = idr/2 #immunity duration for V(days) 
    σS = 1/100 #noise for S
    σE = 1/100 #noise for E
    σI = 1/100 #noise for I
    σR = 1/100 #noise for R
    σV = 1/20 #noise for V
    function seirv2(du, u, p, t)
        S,E,I,R,V = u
        beta, lp, ip, ve, vr, σS, σE, σI, σR, σV = p
        du[1] = -beta*S*I - vr + (1/idr)*R + (1/idv)*V
        du[2] = beta*S*I - (1/lp)*E + (1-ve)*beta*V*I
        du[3] = (1/lp)*E - (1/ip)*I
        du[4] = (1/ip)*I - (1/idr)*R
        du[5] = vr - (1-ve)*beta*V*I - (1/idv)*V
    end
    function seirv_noise(du, u, p, t)
        S,E,I,R,V = u
        beta, lp, ip, ve, vr, σS, σE, σI, σR, σV = p
        du[1] = S*σS
        du[2] = E*σE
        du[3] = I*σI
        du[4] = R*σR
        du[5] = vr*σV
    end
end

In [4]:
@everywhere seirv_prob_st_grad = SDEProblem(seirv2,seirv_noise,[N-E0, E0, 0, 0, 0],(0.0,10.0), [ beta, lp, ip, ve, vr, σS, σE, σI, σR, σV])

In [5]:
@everywhere function jacobian_Matrix_Zy(x,t;seed = 0)
    if seed == 0
        _probFW = remake(seirv_prob_st_grad,tspan = (0.0,t))
    else
        _probFW = remake(seirv_prob_st_grad,tspan = (0.0,t),seed = seed)
    end
    function jacobian(x)
        solve(_probFW,rtol=1e-6,atol=1e-6,saveat=0.25,u0=x[1:5],p=x[6:end],save_everystep=false)[end]
    end
    jacob, = Zygote.jacobian(jacobian,x)[1]
end

In [6]:
@everywhere function jacobain_series(x,tmin=0, tmax=10, saveat=1;seed=0)
    jacobs=[]
    function jacobian_Matrix(t;seed = 0)
        if seed == 0
            _probFW = remake(seirv_prob_st_grad,tspan = (0.0,t))
        else
            _probFW = remake(seirv_prob_st_grad,tspan = (0.0,t),seed = seed)
        end
        function jacobian(x)
            solve(_probFW,rtol=1e-6,atol=1e-6,saveat=0.25,u0=x[1:5],p=x[6:end],save_everystep=false)[end]
        end
        jacob, = Zygote.jacobian(jacobian,x)[1]
    end
    jacobs = tmin:saveat:tmax
    pmap(jacobian_Matrix,jacobs)
end

In [16]:
@everywhere function jacobain_series_FW(x,tmin=0, tmax=10, saveat=1;seed=0)
    jacobs=[]
    function jacobian_Matrix(t;seed = 0)
        if seed == 0
            _probFW = remake(seirv_prob_st_grad,tspan = (0.0,t))
        else
            _probFW = remake(seirv_prob_st_grad,tspan = (0.0,t),seed = seed)
        end
        function jacobian(x)
            solve(_probFW,rtol=1e-6,atol=1e-6,saveat=0.25,u0=x[1:5],p=x[6:end],save_everystep=false,sensealg=ForwardDiffSensitivity())[end]
        end
        jacob = ForwardDiff.jacobian(jacobian,x)
    end
    jacobs = tmin:saveat:tmax
    pmap(jacobian_Matrix,jacobs)
end

In [8]:
@benchmark jacobain_series([N-E0, E0, 0, 0, 0,beta, lp, ip, ve, vr, σS, σE, σI, σR, σV],0,30;seed=1)

BenchmarkTools.Trial: 3 samples with 1 evaluation.
 Range (min … max):  1.408 s …    2.346 s  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.345 s               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.033 s ± 541.621 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁                                                        █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.41 s         Histogram: frequency by time         2.35 s <

 Memory estimate: 150.11 KiB, allocs estimate: 2603.

In [18]:
@benchmark jacobain_series_FW([N-E0, E0, 0, 0, 0,beta, lp, ip, ve, vr, σS, σE, σI, σR, σV],1,30;seed=1)

BenchmarkTools.Trial: 24 samples with 1 evaluation.
 Range (min … max):  179.846 ms … 236.397 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     212.609 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   212.154 ms ±  13.573 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁              ▁ ▁▁ █      ▁   █▁█  █  ▁▁▁▁  ▁ ▁  ▁ ▁     ▁ ▁  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁██▁█▁▁▁▁▁▁█▁▁▁███▁▁█▁▁████▁▁█▁█▁▁█▁█▁▁▁▁▁█▁█ ▁
  180 ms           Histogram: frequency by time          236 ms <

 Memory estimate: 143.92 KiB, allocs estimate: 2486.

In [7]:
using CSV,DataFrames

In [13]:
jacobs = jacobain_series([N-E0, E0, 0, 0, 0,beta, lp, ip, ve, vr, σS, σE, σI, σR, σV],0,30;seed=1)
nothing

In [19]:
dfh = DataFrame(S=[],E=[],I=[],R=[],V=[])
CSV.write("jacabians.csv", dfh,bom=true)
for i in 1:length(jacobs)
    df = DataFrame(S = jacobs[i][1,:],E = jacobs[i][2,:],I = jacobs[i][3,:],R = jacobs[i][4,:],V = jacobs[i][5,:])
    CSV.write("jacabians.csv", df,append=true,bom=true)
end

In [20]:
for i in 1:100
    jacobs = jacobain_series([N-E0, E0, 0, 0, 0,beta, lp, ip, ve, vr, σS, σE, σI, σR, σV],0,30)
    for i in 1:length(jacobs)
        df = DataFrame(S = jacobs[i][1,:],E = jacobs[i][2,:],I = jacobs[i][3,:],R = jacobs[i][4,:],V = jacobs[i][5,:])
        CSV.write("jacabians.csv", df,append=true,bom=true)
    end
end

In [ ]:
grad_S=[]
grad_E=[]
grad_I=[]
grad_R=[]
grad_V=[]
for i in 1:length(jacobs)
    push!(grad_S,jacobs[i][1,:])
    push!(grad_E,jacobs[i][2,:])
    push!(grad_I,jacobs[i][3,:])
    push!(grad_R,jacobs[i][4,:])
    push!(grad_V,jacobs[i][5,:])
end

In [ ]:
using Calculus


In [ ]:
function grad_Matrix_Cal(x,t)
    grad = [([]),([]),([]),([]),([])]
    _probFW = remake(seirv_prob_st_grad,tspan = (0.0,t))
    function grad_S(x)
        solve(_probFW,rtol=1e-6,atol=1e-6,saveat=0.25,u0=x[1:5],p=x[6:end],save_everystep=false)[end][1]
    end
    function grad_E(x)
        solve(_probFW,rtol=1e-6,atol=1e-6,saveat=0.25,u0=x[1:5],p=x[6:end],save_everystep=false)[end][2]
    end
    function grad_I(x)
        solve(_probFW,rtol=1e-6,atol=1e-6,saveat=0.25,u0=x[1:5],p=x[6:end],save_everystep=false)[end][3]
    end
    function grad_R(x)
        solve(_probFW,rtol=1e-6,atol=1e-6,saveat=0.25,u0=x[1:5],p=x[6:end],save_everystep=false)[end][4]
    end
    function grad_V(x)
        solve(_probFW,rtol=1e-6,atol=1e-6,saveat=0.25,u0=x[1:5],p=x[6:end],save_everystep=false)[end][5]
    end
    grad[1], = Zygote.gradient(grad_S,x)
    grad[2], = Zygote.gradient(grad_E,x)
    grad[3], = Zygote.gradient(grad_I,x)
    grad[4], = Zygote.gradient(grad_R,x)
    grad[5], = Zygote.gradient(grad_V,x)
    grad
end